In [22]:
# import mlflow
from lib.constants import PROJECT_DIR, EXPERIMENT_NAME, MLFLOW_URI
from lib.dataset import load_train_data, load_test_data

# Make sure to have the MLFlow server on before running this code.
# mlflow.set_tracking_uri(uri=MLFLOW_URI)
# experiment = mlflow.set_experiment(EXPERIMENT_NAME)
X_train, y_train = load_train_data()
X_test = load_test_data()

In [23]:
import pandas as pd
from lib.sklearn.preprocess import nlp
from sklearn.pipeline import Pipeline

max_words = 750
custom_map = {
    row['asal']: row['tujuan']
    for _, row in pd.read_csv('custom-mapper.csv').iterrows()
}
preprocess_pipeline = Pipeline([
    ('tokenizer', nlp.TextTokenizer()),
    ('formalizer', nlp.WordsFormalizer()),
    ('custom_mapper', nlp.WordsMapper(custom_map)),
    ('lemmatization', nlp.WordsLemmatization()),
    ('special_char_filter', nlp.SpecialCharacterFilter()),
    ('unknown_words_filter', nlp.UnknownWordsFilter()),
    ('text_to_sequence', nlp.TokenSequenceTransformer(max_words=max_words))
])

preprocess_pipeline.fit(X_train + X_test)
X_train_transformed = preprocess_pipeline.transform(X_train)
X_test_transformed = preprocess_pipeline.transform(X_test)
X_train_transformed[:5]

[[4, 189, 3, 259, 245, 343, 329],
 [6, 16, 4, 30, 330, 1, 2, 260, 162, 379],
 [4, 16, 13, 21, 68],
 [5, 14, 1, 38],
 [4, 30, 1, 53, 119, 2, 75, 55]]

In [4]:
min_len = 10
max_len = 20

X_train_seq, y_train_seq = nlp.split_sequences(
    X_train_transformed,
    y=y_train,
    max_len=max_len,
    min_len=min_len
)
X_test_seq = nlp.split_sequences(
    X_test_transformed,
    max_len=max_len,
    min_len=min_len
)

len(X_train_seq)

8679

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import tensorflow as tf
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2024-03-12 21:50:00.525238: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-03-12 21:50:00.527026: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-03-12 21:50:02.080091: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-12 21:50:02.080155: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (blackbox): /proc/driver/nvidia/version does not exist


In [269]:
import numpy as np
import tensorflow.keras.backend as K


def f1_macro(y_true, y_pred):
    y_true = K.cast(y_true, 'float')
    y_pred = K.cast(y_pred, 'float')

    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum(K.max(K.clip(y_pred - y_true, 0, 1), axis=0), axis=0)
    fn = K.sum(K.max(K.clip(y_true - y_pred, 0, 1), axis=0), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    return K.mean(f1)

def f1_macro_loss(y_true, y_pred):
    return 1 - f1_macro(y_true, y_pred)

In [273]:
from lib.sklearn.model.classifier import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Dense, Embedding, LSTM, Maximum

model = Sequential([
    Embedding(input_dim=max_words, output_dim=32, input_length=max_len),
    Bidirectional(LSTM(64)),
    Dense(5, activation='sigmoid')
])

model_pipeline = KerasClassifier(
    model,
    optimizer='adam',
    # loss='categorical_crossentropy',
    # metrics=['accuracy'],
    loss=f1_macro_loss,
    metrics=[f1_macro],
    batch_size=64,
    epochs=5
)

In [274]:
model_pipeline.fit(X_train_seq, y_train_seq)

Epoch 1/5


ValueError: in user code:

    /home/yoiqsram/miniconda3/envs/midnight-ops/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/yoiqsram/miniconda3/envs/midnight-ops/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/yoiqsram/miniconda3/envs/midnight-ops/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/yoiqsram/miniconda3/envs/midnight-ops/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/yoiqsram/miniconda3/envs/midnight-ops/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replic

In [270]:
labels = np.eye(len(np.unique(y_train_seq)))
y_true = np.array([labels[yi - 1] for yi in y_train_seq])

y_pred = model_pipeline.predict_proba(X_train_seq)
f1_macro(y_true, y_pred)

<tf.Tensor: shape=(), dtype=float32, numpy=0.9908126>

In [255]:
y_true

array([[1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]])

In [257]:
y_pred

array([[0.99997675, 0.99998736, 0.99999344, 0.99998724, 0.9999808 ],
       [0.99997675, 0.9999875 , 0.9999933 , 0.9999873 , 0.9999809 ],
       [0.99997675, 0.9999874 , 0.99999344, 0.99998736, 0.9999809 ],
       ...,
       [0.9999768 , 0.9999875 , 0.99999344, 0.9999873 , 0.9999809 ],
       [0.9999768 , 0.9999875 , 0.9999933 , 0.99998736, 0.9999809 ],
       [0.99997675, 0.9999875 , 0.99999344, 0.9999873 , 0.99998087]],
      dtype=float32)

In [ ]:
y_true = K.cast(y_true, 'float')
y_pred = K.cast(y_pred, 'float')
# y_pred = one_hot(y_pred)

tp = K.sum(y_true * y_pred, axis=0)
fp = K.sum(K.max(K.clip(y_pred - y_true, 0, 1), axis=0), axis=0)
fn = K.sum(K.max(K.clip(y_true - y_pred, 0, 1), axis=0), axis=0)

p = tp / (tp + fp + K.epsilon())
r = tp / (tp + fn + K.epsilon())

f1 = 2 * p * r / (p + r + K.epsilon())

In [237]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_test_seq = pad_sequences(
    X_test_transformed,
    maxlen=max_len
)

predictions = model_pipeline.predict(X_test_seq)
submission = pd.DataFrame({'ID': np.arange(len(predictions)), 'LABEL': predictions})
display(submission)
submission.to_csv('test_submission.csv', index=False)

,ID,LABEL
0,0,3
1,1,3
2,2,3
3,3,3
4,4,3
...,...,...
495,495,3
496,496,3
497,497,3
498,498,3


In [238]:
from collections import Counter

Counter(predictions)

Counter({3: 500})